## Загрузка данных <a name="data-loader"></a>

In [1]:
import numpy as np

In [2]:
# используем пакет https://pypi.org/project/rs-datasets/
!pip install rs_datasets

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from rs_datasets import MovieLens

data = MovieLens("100k")
data.info()

ratings


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Toy%20Story%2...,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2,GoldenEye (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?GoldenEye%20(...,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,3,Four Rooms (1995),01-Jan-1995,http://us.imdb.com/M/title-exact?Four%20Rooms%...,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [4]:
users_count_from_log = data.ratings['user_id'].unique().shape 
items_count_from_log = data.ratings['item_id'].unique().shape

In [5]:
# загрузим данные в Spark

from replay.data_preparator import DataPreparator

log = DataPreparator().transform(
    data=data.ratings,
    columns_names={
        "user_id": "user_id",
        "item_id": "item_id",
        "timestamp": "timestamp",
        "relevance": "rating"
    }
).cache()

## Сценарий с разными моделями

In [6]:
import numpy as np
import replay

In [7]:
# import importlib
# importlib.reload(replay)

<module 'replay' from '/mnt2/volodkevich/replay_tasks/replay/replay/__init__.py'>

## Пример с KNN

In [8]:
from replay.models import *

MODELS = {'knn_': KNN(), 
         'slim_': SLIM(),
         'random_rec_': RandomRec()}

PARAM_GRID = {'knn_':
              {
                  'num_neighbours': [3, 100],
                  'shrink': []
              },
              'slim_':
              {
                  'beta': [2**(-8), 2], 
                  'lambda_': [2**(-8), 2]
              },
              'random_rec_':
              {
                  'distribution': ['popular_based', 'uniform']
              }
             }

##  Схема валидации

In [9]:
from replay.splitters import UserSplitter

user_random_splitter = UserSplitter(
    item_test_size=1,
    user_test_size=0.2,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
)

## Обучение сценария <a name="fit-scenario"></a>

In [12]:
from replay.scenarios import MainScenario
from replay.metrics import NDCG, HitRate

### KNN

In [13]:
KNN._search_space['shrink']['type'] = 'int'
KNN._search_space['shrink']['args'] = [0, 50]
KNN._search_space

{'num_neighbours': {'type': 'int', 'args': [5, 100]},
 'shrink': {'type': 'int', 'args': [0, 50]}}

In [15]:
k = 10
k_for_metrics = [10, 5, 1] 
budget = 20
metrics={
        NDCG: k_for_metrics,
        HitRate: k_for_metrics
}

In [16]:
scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=MODELS['knn_'],
    criterion=NDCG,
    metrics=metrics
)

In [17]:
best_knn_params = scenario.research(PARAM_GRID['knn_'],
    log,
    k=k,
    n_trials=budget
)
knn_results = scenario.experiment.results

11-Nov-20 16:55:29, replay, DEBUG: Деление лога на обучающую и тестовую выборку
11-Nov-20 16:55:40, replay, DEBUG: Длина трейна и теста: 99812 187
11-Nov-20 16:55:42, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
11-Nov-20 16:55:43, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 154
11-Nov-20 16:55:44, replay, DEBUG: Инициализация метрик
11-Nov-20 16:55:44, replay, DEBUG: Обучение и предсказание дополнительной модели
11-Nov-20 16:55:44, replay, DEBUG: Начало обучения PopRec
11-Nov-20 16:55:44, replay, DEBUG: Предварительная стадия обучения (pre-fit)
11-Nov-20 16:55:46, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 16:55:48, replay, DEBUG: Начало предикта PopRec
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserW

{10: 0.10019754834638137}


11-Nov-20 16:56:44, replay, DEBUG: NDCG=0.10
[I 2020-11-11 16:56:44,830] Finished trial#0 resulted in value: 0.10019754834638137. Current best value is 0.10019754834638137 with parameters: {'num_neighbours': 82, 'shrink': 40}.
11-Nov-20 16:56:44, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 16:56:44, replay, DEBUG: Начало обучения KNN
11-Nov-20 16:56:44, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 16:56:46, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 16:56:46, replay, DEBUG: Начало предикта KNN
11-Nov-20 16:56:51, replay, DEBUG: -- Подсчет метрики в оптимизации


{10: 0.11867968544793812}


11-Nov-20 16:57:22, replay, DEBUG: NDCG=0.12
[I 2020-11-11 16:57:22,583] Finished trial#1 resulted in value: 0.11867968544793812. Current best value is 0.11867968544793812 with parameters: {'num_neighbours': 68, 'shrink': 25}.
11-Nov-20 16:57:22, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 16:57:22, replay, DEBUG: Начало обучения KNN
11-Nov-20 16:57:22, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 16:57:24, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 16:57:24, replay, DEBUG: Начало предикта KNN
11-Nov-20 16:57:28, replay, DEBUG: -- Подсчет метрики в оптимизации


...

11-Nov-20 17:06:08, replay, DEBUG: NDCG=0.10
[I 2020-11-11 17:06:08,397] Finished trial#17 resulted in value: 0.09551792200282885. Current best value is 0.1633295519305541 with parameters: {'num_neighbours': 12, 'shrink': 2}.
11-Nov-20 17:06:08, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:06:08, replay, DEBUG: Начало обучения KNN
11-Nov-20 17:06:08, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:06:09, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:06:09, replay, DEBUG: Начало предикта KNN
11-Nov-20 17:06:13, replay, DEBUG: -- Подсчет метрики в оптимизации


{10: 0.14133172213480003}


11-Nov-20 17:06:41, replay, DEBUG: NDCG=0.14
[I 2020-11-11 17:06:41,572] Finished trial#18 resulted in value: 0.14133172213480003. Current best value is 0.1633295519305541 with parameters: {'num_neighbours': 12, 'shrink': 2}.
11-Nov-20 17:06:41, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:06:41, replay, DEBUG: Начало обучения KNN
11-Nov-20 17:06:41, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:06:42, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:06:42, replay, DEBUG: Начало предикта KNN
11-Nov-20 17:06:46, replay, DEBUG: -- Подсчет метрики в оптимизации


{10: 0.15605298651709917}


11-Nov-20 17:07:14, replay, DEBUG: NDCG=0.16
[I 2020-11-11 17:07:14,367] Finished trial#19 resulted in value: 0.15605298651709917. Current best value is 0.1633295519305541 with parameters: {'num_neighbours': 12, 'shrink': 2}.
11-Nov-20 17:07:14, replay, DEBUG: Лучшее значение метрики: 0.16
11-Nov-20 17:07:14, replay, DEBUG: Лучшие параметры: {'num_neighbours': 12, 'shrink': 2}


In [18]:
best_knn_params

{'num_neighbours': 12, 'shrink': 2}

### SLIM

In [19]:
scenario.recommender=MODELS['slim_']

best_slim_params = scenario.research(PARAM_GRID['slim_'],
    log,
    k=k,
    n_trials=budget
)
slim_results = scenario.experiment.results

11-Nov-20 17:07:14, replay, DEBUG: Деление лога на обучающую и тестовую выборку
11-Nov-20 17:07:15, replay, DEBUG: Длина трейна и теста: 99812 187
11-Nov-20 17:07:15, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
11-Nov-20 17:07:15, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 154
11-Nov-20 17:07:16, replay, DEBUG: Инициализация метрик
11-Nov-20 17:07:16, replay, DEBUG: Обучение и предсказание дополнительной модели
11-Nov-20 17:07:16, replay, DEBUG: Начало обучения PopRec
11-Nov-20 17:07:16, replay, DEBUG: Предварительная стадия обучения (pre-fit)
11-Nov-20 17:07:16, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:07:17, replay, DEBUG: Начало предикта PopRec
11-Nov-20 17:07:20, replay, DEBUG: Оптимизация параметров
11-Nov-20 17:07:20, replay, DEBUG: Количество попыток: 20
11-Nov-20 17:07:20, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:07:20, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:07:20, replay, DEBUG: Предварите

{10: 0.11256621768601842}


11-Nov-20 17:07:48, replay, DEBUG: NDCG=0.11
[I 2020-11-11 17:07:48,384] Finished trial#0 resulted in value: 0.11256621768601842. Current best value is 0.11256621768601842 with parameters: {'beta': 0.06247182931704085, 'lambda_': 1.7882240290577447}.
11-Nov-20 17:07:48, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:07:48, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:07:48, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:07:49, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:07:49, replay, DEBUG: Начало предикта SLIM
11-Nov-20 17:07:53, replay, DEBUG: -- Подсчет метрики в оптимизации
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packag

{10: 0.24875643630814828}


11-Nov-20 17:08:15, replay, DEBUG: NDCG=0.25
[I 2020-11-11 17:08:15,567] Finished trial#1 resulted in value: 0.24875643630814828. Current best value is 0.24875643630814828 with parameters: {'beta': 0.0048078597244356445, 'lambda_': 0.05876956809966476}.
11-Nov-20 17:08:15, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:08:15, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:08:15, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:08:16, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:08:16, replay, DEBUG: Начало предикта SLIM
11-Nov-20 17:08:20, replay, DEBUG: -- Подсчет метрики в оптимизации
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-pac

...

11-Nov-20 17:15:23, replay, DEBUG: NDCG=0.25
[I 2020-11-11 17:15:23,377] Finished trial#17 resulted in value: 0.25258825358218273. Current best value is 0.2743165952393946 with parameters: {'beta': 1.8799729122662445, 'lambda_': 0.016186199993327346}.
11-Nov-20 17:15:23, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:15:23, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:15:23, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:15:24, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:15:24, replay, DEBUG: Начало предикта SLIM
11-Nov-20 17:15:28, replay, DEBUG: -- Подсчет метрики в оптимизации
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packa

{10: 0.266605038266237}


11-Nov-20 17:15:49, replay, DEBUG: NDCG=0.27
[I 2020-11-11 17:15:49,478] Finished trial#18 resulted in value: 0.266605038266237. Current best value is 0.2743165952393946 with parameters: {'beta': 1.8799729122662445, 'lambda_': 0.016186199993327346}.
11-Nov-20 17:15:49, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:15:49, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:15:49, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:15:50, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:15:50, replay, DEBUG: Начало предикта SLIM
11-Nov-20 17:15:54, replay, DEBUG: -- Подсчет метрики в оптимизации
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-package

{10: 0.27443151905234003}


11-Nov-20 17:16:14, replay, DEBUG: NDCG=0.27
[I 2020-11-11 17:16:15,105] Finished trial#19 resulted in value: 0.27443151905234003. Current best value is 0.27443151905234003 with parameters: {'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}.
11-Nov-20 17:16:15, replay, DEBUG: Лучшее значение метрики: 0.27
11-Nov-20 17:16:15, replay, DEBUG: Лучшие параметры: {'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}


In [20]:
all_results = knn_results.append(slim_results)

In [21]:
best_slim_params

{'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}

### RandomRec (чтобы проверить категориальный параметр)

In [22]:
RandomRec._search_space = dict()
RandomRec._search_space['distribution'] = {'type': 'categorical', 'args': ['popular_based', 'uniform']}

In [36]:
scenario.recommender=MODELS['random_rec_']

best_rr_params = scenario.research(PARAM_GRID['random_rec_'],
    log,
    k=k,
    n_trials=2
)
rr_results = scenario.experiment.results

11-Nov-20 17:23:53, replay, DEBUG: Деление лога на обучающую и тестовую выборку
11-Nov-20 17:23:55, replay, DEBUG: Длина трейна и теста: 99812 187
11-Nov-20 17:23:55, replay, DEBUG: Количество пользователей в трейне и тесте: 943, 187
11-Nov-20 17:23:56, replay, DEBUG: Количество объектов в трейне и тесте: 1681, 154
11-Nov-20 17:23:56, replay, DEBUG: Инициализация метрик
11-Nov-20 17:23:56, replay, DEBUG: Обучение и предсказание дополнительной модели
11-Nov-20 17:23:56, replay, DEBUG: Начало обучения PopRec
11-Nov-20 17:23:56, replay, DEBUG: Предварительная стадия обучения (pre-fit)
11-Nov-20 17:23:56, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:23:57, replay, DEBUG: Начало предикта PopRec
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserW

{10: 0.03371643897890196}


11-Nov-20 17:24:38, replay, DEBUG: NDCG=0.03
[I 2020-11-11 17:24:38,501] Finished trial#0 resulted in value: 0.03371643897890196. Current best value is 0.03371643897890196 with parameters: {'distribution': 'popular_based'}.
11-Nov-20 17:24:38, replay, DEBUG: -- Второй фит модели в оптимизации
11-Nov-20 17:24:38, replay, DEBUG: Начало обучения RandomRec
11-Nov-20 17:24:38, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:24:41, replay, DEBUG: -- Предикт модели в оптимизации
11-Nov-20 17:24:41, replay, DEBUG: Начало предикта RandomRec
11-Nov-20 17:24:50, replay, DEBUG: -- Подсчет метрики в оптимизации


{10: 0.030741882540491386}


11-Nov-20 17:25:19, replay, DEBUG: NDCG=0.03
[I 2020-11-11 17:25:19,555] Finished trial#1 resulted in value: 0.030741882540491386. Current best value is 0.03371643897890196 with parameters: {'distribution': 'popular_based'}.
11-Nov-20 17:25:19, replay, DEBUG: Лучшее значение метрики: 0.03
11-Nov-20 17:25:19, replay, DEBUG: Лучшие параметры: {'distribution': 'popular_based'}


In [37]:
all_results = all_results.append(rr_results)

In [38]:
best_rr_params

{'distribution': 'popular_based'}

## Результаты

In [39]:
all_results.sort_values('NDCG@10', ascending=False)

,HitRate@1,HitRate@5,HitRate@10,NDCG@10,NDCG@1,NDCG@5
"SLIM{'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}",0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
best_slim_scenario,0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
"SLIM{'beta': 1.8799729122662445, 'lambda_': 0.016186199993327346}",0.106952,0.347594,0.491979,0.274317,0.106952,0.227824
"SLIM{'beta': 1.8851075394684569, 'lambda_': 0.01342528450135674}",0.106952,0.342246,0.497326,0.273893,0.106952,0.224450
"SLIM{'beta': 1.663127403505271, 'lambda_': 0.019312079434170928}",0.106952,0.347594,0.491979,0.273813,0.106952,0.227590
"SLIM{'beta': 1.9106522803590236, 'lambda_': 0.019999182491868488}",0.101604,0.347594,0.481283,0.271695,0.101604,0.228423
"SLIM{'beta': 0.4455599911925877, 'lambda_': 0.10152327312240496}",0.117647,0.347594,0.454545,0.270219,0.117647,0.236580
"SLIM{'beta': 0.8732149707670739, 'lambda_': 0.10803703487597296}",0.106952,0.336898,0.459893,0.266605,0.106952,0.227561
"SLIM{'beta': 1.0767151647010165, 'lambda_': 0.004001558568600958}",0.096257,0.315508,0.475936,0.256823,0.096257,0.204856
"SLIM{'beta': 0.16259395873118712, 'lambda_': 0.046553318221315206}",0.112299,0.304813,0.449198,0.252588,0.112299,0.205515


## Проверка, что production работает ок

In [27]:
from replay.experiment import Experiment

In [28]:
train, test = user_random_splitter.split(log)
e = Experiment(test, {NDCG(): k_for_metrics, HitRate(): k_for_metrics})

### Предсказание с помощью сценария

In [29]:
best_slim_params

{'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}

In [30]:
scenario.recommender=MODELS['slim_']

recs = scenario.production(
    best_slim_params, 
    train,
    users=test.select("user_id").distinct().cache(), 
    items=test.select("item_id").distinct().cache(),
    k=10
)

e.add_result('best_slim_scenario', recs)

11-Nov-20 17:17:08, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:17:08, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:17:10, replay, DEBUG: Начало предикта SLIM
11-Nov-20 17:17:11, replay, DEBUG: Начало обучения PopRec
11-Nov-20 17:17:11, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:17:12, replay, DEBUG: Начало предикта PopRec
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [31]:
scenario.recommender=MODELS['knn_']

recs = scenario.production(
    best_knn_params, 
    train,
    users=test.select("user_id").distinct().cache(), 
    items=test.select("item_id").distinct().cache(),
    k=10
)

e.add_result('best_knn_scenario', recs)

11-Nov-20 17:17:32, replay, DEBUG: Начало обучения KNN
11-Nov-20 17:17:32, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:17:33, replay, DEBUG: Начало предикта KNN
11-Nov-20 17:17:35, replay, DEBUG: Начало обучения PopRec
11-Nov-20 17:17:35, replay, DEBUG: Основная стадия обучения (fit)
11-Nov-20 17:17:35, replay, DEBUG: Начало предикта PopRec


In [32]:
all_results = all_results.append(e.results)

### Предсказание чистой моделью

In [33]:
best_slim_model = SLIM(**best_slim_params)
slim_pred = best_slim_model.fit_predict(train, 
                                        users=test.select("user_id").distinct().cache(),
                                        items=test.select("item_id").distinct().cache(),
                                        k=10)

11-Nov-20 17:17:58, replay, DEBUG: Начало обучения SLIM
11-Nov-20 17:17:58, replay, DEBUG: Предварительная стадия обучения (pre-fit)
11-Nov-20 17:17:59, replay, DEBUG: Основная стадия обучения (fit)
/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)
11-Nov-20 17:18:00, replay, DEBUG: Начало предикта SLIM


In [34]:
e.add_result('best_slim_model', slim_pred)

/mnt2/volodkevich/replay_tasks/replay/venv376/lib/python3.7/site-packages/pyspark/sql/pandas/group_ops.py:76: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  "more details.", UserWarning)


In [35]:
all_results.append(e.results).sort_values('NDCG@10', ascending=False)

,HitRate@1,HitRate@5,HitRate@10,NDCG@10,NDCG@1,NDCG@5
best_slim_model,0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
best_slim_scenario,0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
best_slim_scenario,0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
"SLIM{'beta': 1.9347834982278882, 'lambda_': 0.011806526371156959}",0.106952,0.347594,0.497326,0.274432,0.106952,0.226848
"SLIM{'beta': 1.8799729122662445, 'lambda_': 0.016186199993327346}",0.106952,0.347594,0.491979,0.274317,0.106952,0.227824
"SLIM{'beta': 1.8851075394684569, 'lambda_': 0.01342528450135674}",0.106952,0.342246,0.497326,0.273893,0.106952,0.224450
"SLIM{'beta': 1.663127403505271, 'lambda_': 0.019312079434170928}",0.106952,0.347594,0.491979,0.273813,0.106952,0.227590
"SLIM{'beta': 1.9106522803590236, 'lambda_': 0.019999182491868488}",0.101604,0.347594,0.481283,0.271695,0.101604,0.228423
"SLIM{'beta': 0.4455599911925877, 'lambda_': 0.10152327312240496}",0.117647,0.347594,0.454545,0.270219,0.117647,0.236580
"SLIM{'beta': 0.8732149707670739, 'lambda_': 0.10803703487597296}",0.106952,0.336898,0.459893,0.266605,0.106952,0.227561


Все ок, метрики при подборе параметров, применении модели в сценарии и просто обучении с лучшими гиперпараметрами совпадают.